In [1]:
import wikipedia 
import transformers
from pathlib import Path 
from transformers import AutoModelForCausalLM, AutoTokenizer
transformers.utils.logging.set_verbosity(transformers.logging.CRITICAL)



# My personal cache directory
cache_dir = Path('/n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache')
data_cache= Path("/n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia")

if not cache_dir.exists():
    cache_dir = None 
if not data_cache.exists():
    data_cache = None 
    

import os
os.environ["HF_HOME"] = str(cache_dir)


os.environ["TRANSFORMERS_CACHE"] = str(cache_dir)
os.environ["HF_DATASETS_CACHE"] = str(cache_dir)
os.environ["HF_HUB_CACHE"] = str(cache_dir)

In [2]:
import torch 
from transformers.utils import logging
logging.set_verbosity_debug()

model_id = 'HuggingFaceH4/zephyr-7b-beta'

#model_id = "mistralai/Mistral-7B-Instruct-v0.3"
print(f"model_id- {model_id}")
print(f"cache_dir - {cache_dir}")   
device = 'cuda:0'
dtype= torch.float32
model = AutoModelForCausalLM.from_pretrained(model_id,
                 
                                             torch_dtype=dtype, cache_dir=cache_dir,)
model = model.to(device)
model.requires_grad_(False)
tokenizer = AutoTokenizer.from_pretrained(model_id, 
                                          use_fast=False, cache_dir=cache_dir)

loading configuration file config.json from cache at /n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache/models--HuggingFaceH4--zephyr-7b-beta/snapshots/892b3d7a7b1cf10c7a701c60881cd93df615734c/config.json
Model config MistralConfig {
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 2,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.1",
  "use_cache": true,
  "vocab_size": 32000
}



model_id- HuggingFaceH4/zephyr-7b-beta
cache_dir - /n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache


Detected torch version: 2.5.1
Detected torch version: 2.5.1
loading weights file model.safetensors from cache at /n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache/models--HuggingFaceH4--zephyr-7b-beta/snapshots/892b3d7a7b1cf10c7a701c60881cd93df615734c/model.safetensors.index.json
Instantiating MistralForCausalLM model under default dtype torch.float32.
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 2
}



Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing MistralForCausalLM.

All the weights of MistralForCausalLM were initialized from the model checkpoint at HuggingFaceH4/zephyr-7b-beta.
If your task is similar to the task the model of the checkpoint was trained on, you can already use MistralForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache/models--HuggingFaceH4--zephyr-7b-beta/snapshots/892b3d7a7b1cf10c7a701c60881cd93df615734c/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2
}

loading file tokenizer.model from cache at /n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache/models--HuggingFaceH4--zephyr-7b-beta/snapshots/892b3d7a7b1cf10c7a701c60881cd93df615734c/tokenizer.model
loading file added_tokens.json from cache at /n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache/models--HuggingFaceH4--zephyr-7b-beta/snapshots/892b3d7a7b

In [3]:
from importlib import reload 

from wiki_rag import wikipedia
reload(wikipedia)
from wiki_rag import rag

import os

from pathlib import Path
data_cache= Path("/n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/text")





for i, article in enumerate(wikipedia.parse_wikiextractor_output(data_cache)):
    print(article['title'])
    print(article.keys())
    
    text = article['text']
    if len(text) < 100:
        continue
    
    # remove any leading or trailing whitespace
    text = text.strip()
    print(len(text))
    print(text[:1000])  # preview first 300 characters
    if i >10:
        break  # remove this to keep going


AccessibleComputing
dict_keys(['id', 'title', 'url', 'text'])
Anarchism
dict_keys(['id', 'title', 'url', 'text'])
45652
Anarchism

Anarchism is a political philosophy and movement that seeks to abolish all institutions that perpetuate authority, coercion, or hierarchy, primarily targeting the state and capitalism. Anarchism advocates for the replacement of the state with stateless societies and voluntary free associations. A historically left-wing movement, anarchism is usually described as the libertarian wing of the socialist movement (libertarian socialism).
Although traces of anarchist ideas are found all throughout history, modern anarchism emerged from the Enlightenment. During the latter half of the 19th and the first decades of the 20th century, the anarchist movement flourished in most parts of the world and had a significant role in workers' struggles for emancipation. Various anarchist schools of thought formed during this period. Anarchists have taken part in several revolu

In [4]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.embeddings.base import Embeddings
from typing import List
import numpy as np
from pathlib import Path 
import json
import os
from wiki_rag import wikipedia
from itertools import islice
from typing import Iterator


# === Helper to batch an iterator ===
def batched(iterable: Iterator, batch_size: int):
    iterator = iter(iterable)
    while batch := list(islice(iterator, batch_size)):
        yield batch

embeddings = rag.ModelEmbeddings(model, tokenizer, device)
vectorstore = None

wiki_generator = wikipedia.parse_wikiextractor_output(data_cache)
counts = 0
batch_size = 4

docs = []
    
for i, d in enumerate(wiki_generator):
    #print(f"Processing article {counts}: {d['title']}")
    if counts > int(10):
        break

    title = d['title']
    url = d['url']
    text = d['text']
    id_ = d.get('id')
    if len(text) < 100:
        continue
    counts +=1
    
    if counts % 25 == 0:
        print(f"Processed {counts} articles so far...")
    
    text = text.strip()
    # abstract is first 3 par
    abstract = "\n".join(text.split("\n")[:3])
    
    doc = Document(page_content=abstract, metadata={"title": title})
    
    
    if vectorstore is None:
        vectorstore = FAISS.from_documents([doc], embeddings)
    else:
        vectorstore.add_documents([doc])

index_path = "wiki_faiss_index"
if vectorstore:
    vectorstore.save_local(index_path)
    print(f"✅ FAISS index saved to {index_path}")
else:
    print("⚠️ No documents were indexed.")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ FAISS index saved to wiki_faiss_index


loading configuration file config.json from cache at /n/home04/rrinberg/.cache/huggingface/hub/models--BAAI--bge-large-en/snapshots/abe7d9d814b775ca171121fb03f394dc42974275/config.json
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.51.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file model.safetensors from cache at /n/home04/rrinberg/.cache/huggingface/hu

In [ ]:

from langchain.embeddings import HuggingFaceEmbeddings

class PromptedBGE(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        return super().embed_documents([
            f"Represent this document for retrieval: {t}" for t in texts
        ])

    def embed_query(self, text):
        return super().embed_query(f"Represent this query for retrieval: {text}")
# BAAI_embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en")

BAAI_embedding = PromptedBGE(model_name="BAAI/bge-base-en")  # or bge-large-en


config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/719 [00:00<?, ?B/s]

loading configuration file config.json from cache at /n/home04/rrinberg/.cache/huggingface/hub/models--BAAI--bge-base-en/snapshots/b737bf5dcc6ee8bdc530531266b4804a5d77b5d8/config.json
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.51.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

loading file vocab.txt from cache at /n/home04/rrinberg/.cache/huggingface/hub/models--BAAI--bge-base-en/snapshots/b737bf5dcc6ee8bdc530531266b4804a5d77b5d8/vocab.txt
loading file tokenizer.json from cache at /n/home04/rrinberg/.cache/huggingface/hub/models--BAAI--bge-base-en/snapshots/b737bf5dcc6ee8bdc530531266b4804a5d77b5d8/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /n/home04/rrinberg/.cache/huggingface/hub/models--BAAI--bge-base-en/snapshots/b737bf5dcc6ee8bdc530531266b4804a5d77b5d8/special_tokens_map.json
loading file tokenizer_config.json from cache at /n/home04/rrinberg/.cache/huggingface/hub/models--BAAI--bge-base-en/snapshots/b737bf5dcc6ee8bdc530531266b4804a5d77b5d8/tokenizer_config.json
loading file chat_template.jinja from cache at None


config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:

#embeddings = rag.ModelEmbeddings(model, tokenizer, device)
vectorstore = None
counts = 0

# get the top 1M articles
HOMEDIR = Path.home()
BASEDIR = HOMEDIR / 'code/wiki-rag'
asset_dir = BASEDIR / 'assets'

json_dir = data_cache / 'json'

output_f = asset_dir / 'english_pageviews.csv'
stats_f = asset_dir / 'pageviews-20241201-000000'
print(f"loading english df from {output_f}")
english_df = wikipedia.get_sorted_english_df(output_f, stats_f) # output - where to output, stats_f base

title_to_file_path_f_pkl = asset_dir / 'title_to_file_path.pkl'
print(f"loading wiki index from {title_to_file_path_f_pkl}")

title_to_file_path = wikipedia.get_title_to_path_index(json_dir, title_to_file_path_f_pkl)


buffer = []
batch_size = 10

from tqdm import tqdm
max_articles = 500


for i, row in enumerate(tqdm(english_df.itertuples(index=False))):

    #print(f"Processing article {counts}: {d['title']}")
    if i > int(max_articles):
        break

    title = row.page_title
    clean_title_ = wikipedia.clean_title(title)
    data = wikipedia.get_wiki_page(clean_title_, title_to_file_path)
    if data is None:
        continue
    
    title = data['title']
    url = data['url']
    text = data['text']
    id_ = data.get('id')
    
    if len(text) < 100:
        continue
    
    counts +=1

    if counts % 250 == 0:
        print(f"Processed {counts} articles so far...")
    
    text = text.strip()
    # abstract is first 3 par
    abstract = "\n".join(text.split("\n")[:1])
    
    doc = Document(page_content=abstract, metadata={"title": title, "ind": i, "url": url, "id": id_})

    #doc = Document(page_content=text, metadata={"title": title, "ind": i, "url": url, "id": id_})

    buffer.append(doc)

    if len(buffer) >= batch_size:
        if vectorstore is None:
            print(f"len buffer - {len(buffer)}")
            with torch.no_grad():
                #vectorstore = FAISS.from_documents(buffer, embeddings)
                vectorstore = FAISS.from_documents(buffer, BAAI_embedding)
        else:
            vectorstore.add_documents(buffer)
        buffer.clear()
if vectorstore is None:
    print(f"len buffer - {len(buffer)}")
    with torch.no_grad():
        vectorstore = FAISS.from_documents(buffer, embeddings)
else:
    vectorstore.add_documents(buffer)
buffer.clear()

# save 
# print out how many
print(f"Total articles processed: {counts}")
print(f"entries in vectorstore: {vectorstore.index.ntotal}")




loading english df from /n/home04/rrinberg/code/wiki-rag/assets/english_pageviews.csv
loaded from /n/home04/rrinberg/code/wiki-rag/assets/english_pageviews.csv
loading wiki index from /n/home04/rrinberg/code/wiki-rag/assets/title_to_file_path.pkl


111it [00:00, 547.79it/s]

len buffer - 10


501it [00:00, 628.82it/s]

Processed 250 articles so far...
Total articles processed: 293
entries in vectorstore: 293


In [23]:
print(abstract)

Smile is a 2022 American supernatural psychological horror film written and directed by Parker Finn, in his feature directorial debut. A follow-up to Parker's short film "Laura Hasn't Slept" (2020), it stars Sosie Bacon as a therapist who witnesses the bizarre suicide of a patient, then becomes overwhelmed with increasingly disturbing and daunting visions that lead her to believe she is experiencing something supernatural. It also features Jessie T. Usher, Kyle Gallner, Kal Penn, and Rob Morgan, as well as Caitlin Stasey reprising her role from "Laura Hasn't Slept".


In [24]:
for doc in vectorstore.docstore._dict.values():
    
    print(doc.metadata)

{'title': 'Wicked (2024 film)', 'ind': 4, 'url': 'https://en.wikipedia.org/wiki?curid=60455274', 'id': '60455274'}
{'title': 'Wicked (musical)', 'ind': 5, 'url': 'https://en.wikipedia.org/wiki?curid=1678965', 'id': '1678965'}
{'title': 'Bob Bryar', 'ind': 6, 'url': 'https://en.wikipedia.org/wiki?curid=50782727', 'id': '50782727'}
{'title': 'Moana 2', 'ind': 7, 'url': 'https://en.wikipedia.org/wiki?curid=65419058', 'id': '65419058'}
{'title': 'Charles Kushner', 'ind': 8, 'url': 'https://en.wikipedia.org/wiki?curid=913005', 'id': '913005'}
{'title': 'Gladiator II', 'ind': 9, 'url': 'https://en.wikipedia.org/wiki?curid=72682130', 'id': '72682130'}
{'title': 'Sunny Edwards', 'ind': 11, 'url': 'https://en.wikipedia.org/wiki?curid=62371618', 'id': '62371618'}
{'title': 'Hailee Steinfeld', 'ind': 12, 'url': 'https://en.wikipedia.org/wiki?curid=26260043', 'id': '26260043'}
{'title': 'Cynthia Erivo', 'ind': 13, 'url': 'https://en.wikipedia.org/wiki?curid=48453494', 'id': '48453494'}
{'title': '

In [27]:
resp = vectorstore.search("football", k=10, search_type="similarity")
resp 
#num_elements = 
#vectorstore.index.ntotal


[Document(id='48d43cca-9874-45ce-81c2-00fd9d9f64d1', metadata={'title': 'College football national championships in NCAA Division I FBS', 'ind': 493, 'url': 'https://en.wikipedia.org/wiki?curid=17278765', 'id': '17278765'}, page_content='A national championship in the highest level of college football in the United States, currently the NCAA Division I Football Bowl Subdivision (FBS), is a designation awarded annually by various organizations to their selection of the best college football team. Division I FBS football is the only National Collegiate Athletic Association (NCAA) sport for which the NCAA does not host a yearly championship event. As such, it is sometimes referred to as a "mythical national championship".'),
 Document(id='14a47dee-1b5b-4f0d-a14d-a06f21bb77df', metadata={'title': '2024–25 NCAA football bowl games', 'ind': 403, 'url': 'https://en.wikipedia.org/wiki?curid=76147383', 'id': '76147383'}, page_content='The 2024–25 NCAA football bowl games were a series of colleg

In [6]:
[a for a in dir(resp[0]) if "__" not in a]
resp[0].metadata

{'title': 'A'}

In [8]:

wiki_generator = wikipedia.wikipedia_abstract_generator(data_cache)

#for _ in range(3):
#    print(next(wiki_generator))